# Explorando a Requests

In [178]:
import requests
from dotenv import load_dotenv 
import os

In [179]:
# Lê o arquivo .env e carrega as variáveis no ambiente do Python
load_dotenv()

True

In [180]:
# Retorna uma lista dos eventos públicos mais recentes do GitHub
r = requests.get('https://api.github.com/events')

In [181]:
r

<Response [200]>

In [182]:
r.url

'https://api.github.com/events'

In [183]:
# Retorna todo conteúdo em uma única string
r.text

'[{"id":"52646463046","type":"PushEvent","actor":{"id":164597742,"login":"satvik1001","display_login":"satvik1001","gravatar_id":"","url":"https://api.github.com/users/satvik1001","avatar_url":"https://avatars.githubusercontent.com/u/164597742?"},"repo":{"id":1027952986,"name":"satvik1001/Tysiee-Game-Zone","url":"https://api.github.com/repos/satvik1001/Tysiee-Game-Zone"},"payload":{"repository_id":1027952986,"push_id":25778651154,"size":1,"distinct_size":1,"ref":"refs/heads/main","head":"eacf1eed0872073a0a5bc015f520407aebaaa1d4","before":"9e96a1223dfa889b1aadecdab56050e41fd0a84e","commits":[{"sha":"eacf1eed0872073a0a5bc015f520407aebaaa1d4","author":{"email":"164597742+satvik1001@users.noreply.github.com","name":"satvik1001"},"message":"Add files via upload","distinct":true,"url":"https://api.github.com/repos/satvik1001/Tysiee-Game-Zone/commits/eacf1eed0872073a0a5bc015f520407aebaaa1d4"}]},"public":true,"created_at":"2025-07-28T19:41:34Z"},{"id":"52646463158","type":"WatchEvent","actor":

In [ ]:
# Retorna todo conteúdo em formato json
r.json()

In [185]:
# Utilizando outro endpoint
r = requests.get('https://api.github.com/users/thaisbsts')
r.status_code

200

In [ ]:
# Mostra as informações sobre o usuário
r.json()

In [187]:
response_json = r.json()

In [188]:
# Extraindo a url que armazena as informações sobre repositórios
repos_url = response_json["repos_url"]

# Extraindo as informações da url de repositórios
response_repos = requests.get(repos_url)

In [189]:
# Tranformando em json e coletando o nº de elementos
repos = response_repos.json()
n_repos = len(repos)

In [190]:
# Extraindo o nome, nome de usuário e data de criação da conta
name = response_json["name"]
username = response_json["login"]
date_created = response_json["created_at"][0:10]

In [191]:
# Imprimindo as informações
print(f"O usuário {name} ({username}) criou sua conta no dia {date_created} e possui {n_repos} repositórios.")

O usuário Thais Bastos (thaisbsts) criou sua conta no dia 2024-06-07 e possui 14 repositórios.


# Obtendo dados dos repositórios

In [192]:
# Especificando a versão da API
headers = {'X-GitHub-Api-Version': '2022-11-28'}

In [193]:
api_base_url = 'https://api.github.com'
owner = 'amzn' # Username de quem vamos extrair os dados (Amazon)
url = f'{api_base_url}/users/{owner}/repos'

In [194]:
# Obtendo lista de repositórios da Amazon
response = requests.get(url, headers=headers)
response.status_code

200

In [195]:
''' 
Apesar do código abaixo retornar o número 30, a Amazon não possui apenas 30 repositórios.
Para acessar todos, é preciso considerar tanto a autenticação quanto a paginação da API.
'''

len(response.json())

30

## Autenticação
Caso eu não faça a autenticação, o limite de requisições por hora é 60.

In [196]:
# Token gerado no Github
access_token = os.getenv('API_KEY')
headers = { 'Authorization': 'Bearer ' + access_token,
           'X-GitHub-Api-Version': '2022-11-28'
           }

In [197]:
# Teste para verificar se está correto
response = requests.get(url, headers=headers)
response.status_code

200

## Paginação

In [198]:
api_base_url = 'https://api.github.com'
owner = 'amzn' # Username de quem vamos extrair os dados (Amazon)
url = f'{api_base_url}/users/{owner}/repos'

repos_list = [] # Lista de listas, cada lista é uma página
for page_num in range(1, 7):
    try:
        url_page = f'{url}?page={page_num}'
        response = requests.get(url_page, headers=headers)
        repos_list.append(response.json())
    except:
        repos_list.append(None)

In [199]:
soma = 0
for i in range(0, 6):
    soma += len(repos_list[i])

print(soma)

161


In [200]:
''' 
Atividade: encontrar a quantidade de seguidores da Amazon.
'''

username = 'amzn'
url = f"https://api.github.com/users/{username}/followers"
followers_list = []
page = 1

''' 
Como o número total de páginas é desconhecido, usarei um while True 
com a condição de parada sendo quando a página retornar zero itens (seguidores).
'''
while True: 
    url_page = f'{url}?page={page}'
    response = requests.get(url_page, headers=headers)
    followers = response.json()

    # Caso a lista esteja vazia, posso sair do laço pois todos os dados foram extraidos
    if len(followers)==0:
        break

    # Adicionando os seguidores a lista
    followers_list.append(followers)

    # Incrementa o valor de 'page' para a próxima requisição
    page += 1


In [201]:
# Somando e imprimindo o resultado
total = sum(len(page) for page in followers_list)
print(f"A Amazon tem {total} seguidores.")

A Amazon tem 2118 seguidores.
